In [82]:
import nltk
from collections import defaultdict, Counter
import gensim, logging
import nltk
from gensim import matutils
from gensim.models.keyedvectors import KeyedVectors
from gensim.similarities.index import AnnoyIndexer
from multiprocessing import Process, Pool
import os
import collections
from random import shuffle
from copy import deepcopy
from collections import OrderedDict
import pickle
import networkx as nx
import matplotlib.pyplot as plt
from operator import itemgetter
from scipy.stats import spearmanr
import pandas as pd
import numpy as np
from gensim.models.wrappers import FastText
import string
from numpy import dot, zeros, dtype, float32 as REAL,\
    double, array, vstack, fromstring, sqrt, newaxis,\
    ndarray, sum as np_sum, prod, ascontiguousarray,\
    argmax

#### Spearman Correlation for different word embeddings on Standford rare word dataset
word2vec: (0.2516646845987402, 1825)

word2vec with morph 100000: (0.24562283855541628, 1988)

glove6B 50dimensions (0.26159587465252532, 1782)

glove6B 50dims with morph 100000 : (0.25914850463800143, 1996) (0.229)

glove 100dimensions (0.2136083090117197, 1782)

glove 200dimensions (0.17667532980978479, 1782)

glove 300dimensions (0.15571816294824803, 1782)

glove 42B 300dimensions 

Fasttext: wiki.en.vec (0.3675364161772991, 1990)

Fasttext:wiki-news-300d-1M.vec (0.29925381437203319, 1966)

In [138]:
%time fasttext =KeyedVectors.load_word2vec_format('/home/raja/models/GoogleNews-vectors-negative300.bin', binary=True)

CPU times: user 25.6 s, sys: 2.45 s, total: 28.1 s
Wall time: 32.4 s


In [121]:
len(fasttext.wv.vocab)

400000

In [122]:
graph_file = '../data/norm_graph_100000_17168'
graph_handle = open(graph_file, "rb")
G = pickle.load(graph_handle)
graph_handle.close()

In [123]:
list(G.edges(keys=True, data=True))[38]

('autistic',
 'autism',
 ('antagonistic', 'antagonism'),
 {'cos': 0.5927902460098267, 'morp_rule': ('suffix', 'tic', 'm'), 'rank': 2})

In [124]:
morph_rules = {}
for edge in G.edges(keys=True, data=True):
    morph_rules[edge[3]['morp_rule']] = edge[2]

In [132]:
def get_spearman_corr(word_vectors):
    rw = pd.read_csv('../dataset/rw/rw.txt', header=None,sep='\t')
    rare_word_pair = []
    for index, row in rw.iterrows():
        rare_word_pair.append((row[0], row[1]))
    corr_list = []
    for word1,word2 in rare_word_pair:
        try:
            cor, p = spearmanr(word_vectors.wv[word1], word_vectors.wv[word2])
            corr_list.append(cor)
        except Exception as e:
            if word1 in word_vectors.vocab:
                match, word2_vec = get_oov_vector(word2)
                if match:
                    cor, p = spearmanr(word_vectors.wv[word1],word2_vec)
                    print (word1,cor)
                    corr_list.append(cor)
            elif word2 in word_vectors.vocab:
                match, word1_vec = get_oov_vector(word1)
                if match:
                    cor, p = spearmanr(word_vectors.wv[word2],word1_vec)
                    print (word2,cor)
                    corr_list.append(cor)
    return np.mean(corr_list), len(corr_list)

In [126]:
def get_oov_vector(word):
    new_word = (False, 0, ())
    mean = []
    for rule in morph_rules:
        candidate_word = apply_rule(rule, word)
        if candidate_word and candidate_word in fasttext.vocab:
            if fasttext.vocab[candidate_word].count > new_word[1]:
                new_word = (candidate_word, fasttext.vocab[candidate_word].count, morph_rules[rule])
    if new_word[0]:
        print (word, new_word)
        mean.append(fasttext[new_word[2][0]] * 1)
        mean.append(fasttext[new_word[2][1]] * -1)
        mean.append(fasttext[new_word[0]] * 1)
        mean = matutils.unitvec(array(mean).mean(axis=0)).astype(REAL)
        return True, mean
    else:
        return False, []
#             print ("new word",new_word, word)

In [127]:
def apply_rule(rule, word):
    if rule[0] == 'suffix':
        if word[::-1].startswith(rule[1][::-1]):
            return word[::-1].replace(rule[1][::-1], rule[2][::-1], 1)[::-1]
    else:
        if word.startswith(rule[1]):
            return word.replace(rule[1], rule[2], 1)
    return False

In [139]:
get_spearman_corr(fasttext)

(0.2258053340180437, 2034)

In [118]:
disjoined = fasttext.wv['disrespect'] - fasttext.wv['respect'] + fasttext.wv['join']